# Features extraction & engineering

In [232]:
import os
import math
import sys

from geopy.distance import vincenty
from geopy.distance import geodesic
from pathlib import Path

Setup directory for local libraty Import in notebook

In [2]:
current_dir = get_ipython().getoutput('pwd')
module_path = Path(current_dir[0]).parent

if module_path not in sys.path:
    sys.path.append(str(module_path))

# import loader for PLT files
import read_geolife

Load PLT files to DataFrame

In [3]:
df = read_geolife.read_all_users('Data')

[1/182] processing user 000
[2/182] processing user 001
[3/182] processing user 002
[4/182] processing user 003
[5/182] processing user 004
[6/182] processing user 005
[7/182] processing user 006
[8/182] processing user 007
[9/182] processing user 008
[10/182] processing user 009
[11/182] processing user 010


C:\Users\philka-ua\geekhubds\project_car\read_geolife.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  points['label'][no_label] = 0


[12/182] processing user 011
[13/182] processing user 012
[14/182] processing user 013
[15/182] processing user 014
[16/182] processing user 015
[17/182] processing user 016
[18/182] processing user 017
[19/182] processing user 018
[20/182] processing user 019
[21/182] processing user 020
[22/182] processing user 021
[23/182] processing user 022
[24/182] processing user 023
[25/182] processing user 024
[26/182] processing user 025
[27/182] processing user 026
[28/182] processing user 027
[29/182] processing user 028
[30/182] processing user 029
[31/182] processing user 030
[32/182] processing user 031
[33/182] processing user 032
[34/182] processing user 033
[35/182] processing user 034
[36/182] processing user 035
[37/182] processing user 036
[38/182] processing user 037
[39/182] processing user 038
[40/182] processing user 039
[41/182] processing user 040
[42/182] processing user 041
[43/182] processing user 042
[44/182] processing user 043
[45/182] processing user 044
[46/182] proce

Size DataFrame

In [224]:
df.shape

(24876978, 6)

In [227]:
df.describe()

,lat,lon,alt,label,user
count,2.487698e+07,2.487698e+07,2.487698e+07,2.487698e+07,2.487698e+07
mean,3.897989e+01,1.123597e+02,5.130172e+02,6.807672e-01,7.660328e+01
std,4.070501e+00,2.717960e+01,2.301976e+03,1.689962e+00,5.672404e+01
min,1.044024e+00,-1.799696e+02,-3.264760e+04,0.000000e+00,0.000000e+00
25%,3.990636e+01,1.163065e+02,7.870000e+01,0.000000e+00,2.500000e+01
50%,3.997498e+01,1.163342e+02,1.476000e+02,0.000000e+00,6.600000e+01
75%,3.999848e+01,1.164194e+02,2.190000e+02,0.000000e+00,1.350000e+02
max,4.001667e+02,1.799969e+02,1.075033e+05,1.100000e+01,1.810000e+02


Labels for DATA

In [6]:
read_geolife.mode_ids

{'walk': 1,
 'bike': 2,
 'bus': 3,
 'car': 4,
 'subway': 5,
 'train': 6,
 'airplane': 7,
 'boat': 8,
 'run': 9,
 'motorcycle': 10,
 'taxi': 11}

Labeled and UnLabeled DATA

In [231]:
print('UnLabeled:',df[(df['label'] == 0) & (df['lat'] < 90)].shape[0])
print('Labeled:', df[(df['label'] != 0) & (df['lat'] < 90)].shape[0])

UnLabeled: 19449861
Labeled: 5427116


Copy Labeled DATA

In [237]:
dff = df[(df['label'] != 0) & (df['lat'] < 90)]
dff.shape

(5427116, 6)


**Features engineering**


**For each gps point**

Time delta

In [239]:
%%time

# Time difference between points
dff['delta_time'] = abs(dff['time'] - dff['time'].shift(-1)).apply(lambda x: x.seconds)
dff['lat_1'] = dff['lat'].shift(-1)
dff['lon_1'] = dff['lon'].shift(-1)

dff = dff.iloc[:-1]
dff.head(5)

,time,lat,lon,alt,label,user,delta_time,lat_1,lon_1
1,2008-03-28 14:54:40,39.894178,116.318200,-777.0,6,10,34.0,39.894505,116.321132
2,2008-03-28 14:55:14,39.894505,116.321132,-777.0,6,10,59.0,39.894953,116.326452
3,2008-03-28 14:56:13,39.894953,116.326452,-777.0,6,10,59.0,39.894600,116.332542
4,2008-03-28 14:57:12,39.894600,116.332542,-777.0,6,10,59.0,39.889622,116.337040
5,2008-03-28 14:58:11,39.889622,116.337040,-777.0,6,10,59.0,39.882090,116.338353


Threshold for trip - number of trip (quantity of PLT files)

In [244]:
# Approximately threshold

print('Threshold: ', dff[dff['delta_time'] > 1800].shape[0])

Threshold:  5840


Functions for features

In [245]:
therthold = 1800
therthold_10 = 66150


def compute_distance(points):
      
    lat_long_1 = (points['lat'], points['lon'])
    lat_long_2 = (points['lat_1'], points['lon_1'])
    
    distance = geodesic(lat_long_1, lat_long_2).meters
    
    if points['delta_time'] > therthold:
        distance = 0 
    
    return distance

def compute_distance_10(points):
      
    lat_long_1 = (points['lat'], points['lon'])
    lat_long_2 = (points['lat_10'], points['lon_10'])
    
    distance = geodesic(lat_long_1, lat_long_2).meters
    
    if points['delta_time_10'] > therthold_10:
        distance = 0 
    
    return distance 

def compute_speed(points):
    
    if points['delta_time'] == 0:
        speed = 0
    else: speed = points['distance'] / points['delta_time']
    
    return speed 

def compute_speed_10(points):
    
    if points['delta_time_10'] == 0:
        speed = 0
    else: speed = points['distance_10'] / points['delta_time_10']
    
    return speed 

def compute_acceleration(points):
    
    if points['delta_time'] == 0:
        acceleration = 0
    else: acceleration = (points['speed_1'] -  points['speed']) / points['delta_time']
    
    return acceleration

def compute_acceleration_10(points):
    
    if points['delta_time_10'] == 0:
        acceleration = 0
    else: acceleration = (points['speed_10'] -  points['speed']) / points['delta_time_10']
    
    return acceleration 

def compute_jerk(points):

    if points['delta_time'] == 0:
        jerk = 0
    else: jerk = (points['acceleration_1'] -  points['acceleration']) / points['delta_time']
    
    return jerk

def compute_jerk_10(points):

    if points['delta_time_10'] == 0:
        jerk = 0
    else: jerk = (points['acceleration_10'] -  points['acceleration']) / points['delta_time_10']
    
    return jerk


def compute_bearing(points):
    y = math.sin(math.radians(points['lon_1']) - math.radians(points['lon'])) * math.radians(math.cos(points['lat_1']))
    x = math.radians(math.cos(points['lat'])) * math.radians(math.sin(points['lat_1'])) - \
        math.radians(math.sin(points['lat'])) * math.radians(math.cos(points['lat_1'])) \
        * math.radians(math.cos(points['lon_1']) - math.radians(points['lon']))
    # Convert radian from -pi to pi to [0, 360] degree
    return (math.atan2(y, x) * 180. / math.pi + 360) % 360

def compute_bearing_10(points):
    y = math.sin(math.radians(points['lon_10']) - math.radians(points['lon'])) * math.radians(math.cos(points['lat_10']))
    x = math.radians(math.cos(points['lat'])) * math.radians(math.sin(points['lat_10'])) - \
        math.radians(math.sin(points['lat'])) * math.radians(math.cos(points['lat_10'])) \
        * math.radians(math.cos(points['lon_10']) - math.radians(points['lon']))
    # Convert radian from -pi to pi to [0, 360] degree
    return (math.atan2(y, x) * 180. / math.pi + 360) % 360


def compute_bearing_rate(points):     
    return abs(points['bearing_1'] -  points['bearing'])

def compute_bearing_rate_10(points):     
    return abs(points['bearing_10'] -  points['bearing'])

def compute_elevation_change(points):      
    return abs(points['alt'] -  points['alt_1'])

def compute_elevation_change_10(points):      
    return abs(points['alt'] -  points['alt_10'])

def compute_elevation_speed(points):    
    return points['elevation_change'] * points['speed']

def compute_elevation_speed_10(points):    
    return points['elevation_change_10'] * points['speed_10']

def compute_elevation_angle(points):
    
    if points['distance'] == 0:
        tan_angle = 0    
    else: tan_angle = abs(points['alt_1'] / points['distance'])
    
    return math.atan(tan_angle)

def compute_elevation_angle_10(points):
    
    if points['distance_10'] == 0:
        tan_angle = 0    
    else: tan_angle = abs(points['alt_10'] / points['distance_10'])
    
    return math.atan(tan_angle)

def compute_elevation_distance(points):       
    return points['elevation_change'] * points['distance']

def compute_elevation_distance_10(points):       
    return points['elevation_change_10'] * points['distance_10']

Distance between points

In [68]:
%%time
# dff['distance'] = dff[['lat','lat_1','lon','lon_1','alt','delta_time']].apply(lambda x: compute_distance(x), axis=1)
dff['distance'] = dff[['lat','lat_1','lon','lon_1','alt','delta_time']].apply(compute_distance, axis=1)

Wall time: 58.6 s


Speed

In [100]:
%%time
dff['speed'] = dff[['distance','delta_time']].apply(compute_speed, axis=1)

Wall time: 6min 10s


Acceleration

In [103]:
dff['speed_1'] = dff['speed'].shift(-1)
dff = dff.iloc[:-1]
dff.head(5)

,time,lat,lon,alt,label,user,delta_time,lat_1,lon_1,distance,speed,speed_1
1,2008-03-28 14:54:40,39.894178,116.318200,-777.0,6,10,34.0,39.894505,116.321132,253.375124,7.452210,7.757681
2,2008-03-28 14:55:14,39.894505,116.321132,-777.0,6,10,59.0,39.894953,116.326452,457.703187,7.757681,8.852857
3,2008-03-28 14:56:13,39.894953,116.326452,-777.0,6,10,59.0,39.894600,116.332542,522.318591,8.852857,11.413939
4,2008-03-28 14:57:12,39.894600,116.332542,-777.0,6,10,59.0,39.889622,116.337040,673.422399,11.413939,14.301759
5,2008-03-28 14:58:11,39.889622,116.337040,-777.0,6,10,59.0,39.882090,116.338353,843.803760,14.301759,15.206567


In [ ]:
%%time
dff['acceleration'] = dff[['speed','speed_1','delta_time']].apply(compute_acceleration, axis=1)

Jerk

In [ ]:
dff['acceleration_1'] = dff['acceleration'].shift(-1)
dff = dff.iloc[:-1]

In [ ]:
%%time
dff['jerk'] = dff[['acceleration','acceleration_1','delta_time']].apply(compute_jerk, axis=1)

Bearing

In [ ]:
%%time
dff['bearing'] = dff[['lat','lat_1','lon','lon_1']].apply(compute_bearing, axis=1)

Bearing rate

In [ ]:
dff['bearing_1'] = dff['bearing'].shift(-1)
dff = dff.iloc[:-1]

In [ ]:
%%time
dff['bearing_rate'] = dff[['bearing_1','bearing']].apply(compute_bearing_rate, axis=1)

Elevation change

In [ ]:
dff['alt_1'] = dff['alt'].shift(-1)
dff = dff.iloc[:-1]

In [ ]:
%%time
dff['elevation_change'] = dff[['alt_1','alt']].apply(compute_elevation_change, axis=1)

Elevation speed

In [ ]:
%%time
dff['elevation_speed'] = dff[['elevation_change','speed']].apply(compute_elevation_speed, axis=1)

Elevation angle (slope)

In [ ]:
%%time
dff['elevation_angle'] = dff[['alt_1','distance']].apply(compute_elevation_angle, axis=1)

Elevation distance

In [ ]:
%%time
dff['elevation_distance'] = dff[['elevation_change','distance']].apply(compute_elevation_distance, axis=1)

In [132]:
dff = dff.iloc[:-1]
dff

,time,lat,lon,alt,label,user,delta_time,lat_1,lon_1,distance,...,acceleration_1,jerk,bearing,bearing_1,bearing_rate,alt_1,elevation_change,elevation_speed,elevation_angle,elevation_distance
1,2008-03-28 14:54:40,39.894178,116.318200,-777.0,6,10,34.0,39.894505,116.321132,253.375124,...,0.018562,0.000282,180.196781,180.357221,1.604400e-01,-777.0,0.0,0.000000,1.255575,0.000000
2,2008-03-28 14:55:14,39.894505,116.321132,-777.0,6,10,59.0,39.894953,116.326452,457.703187,...,0.043408,0.000421,180.357221,180.408402,5.118119e-02,-777.0,0.0,0.000000,1.038456,0.000000
3,2008-03-28 14:56:13,39.894953,116.326452,-777.0,6,10,59.0,39.894600,116.332542,522.318591,...,0.048946,0.000094,180.408402,180.298782,1.096195e-01,-777.0,0.0,0.000000,0.978956,0.000000
4,2008-03-28 14:57:12,39.894600,116.332542,-777.0,6,10,59.0,39.889622,116.337040,673.422399,...,0.015336,-0.000570,180.298782,180.086457,2.123256e-01,-777.0,0.0,0.000000,0.856689,0.000000
5,2008-03-28 14:58:11,39.889622,116.337040,-777.0,6,10,59.0,39.882090,116.338353,843.803760,...,0.019319,0.000068,180.086457,180.006671,7.978598e-02,-777.0,0.0,0.000000,0.744205,0.000000
6,2008-03-28 14:59:10,39.882090,116.338353,-777.0,6,10,60.0,39.873873,116.338455,912.394041,...,0.034649,0.000255,180.006671,179.974212,3.245883e-02,-777.0,0.0,0.000000,0.705426,0.000000
7,2008-03-28 15:00:10,39.873873,116.338455,-777.0,6,10,59.0,39.865182,116.338058,965.577796,...,0.044349,0.000164,179.974212,179.977582,3.369977e-03,-777.0,0.0,0.000000,0.677600,0.000000
8,2008-03-28 15:01:09,39.865182,116.338058,-777.0,6,10,59.0,39.855403,116.337710,1086.189677,...,0.085104,0.000691,179.977582,179.786256,1.913256e-01,-777.0,0.0,0.000000,0.620950,0.000000
9,2008-03-28 15:02:08,39.855403,116.337710,-777.0,6,10,59.0,39.844532,116.334362,1240.567528,...,0.043109,-0.000712,179.786256,179.605540,1.807161e-01,-777.0,0.0,0.000000,0.559552,0.000000
10,2008-03-28 15:03:07,39.844532,116.334362,-777.0,6,10,59.0,39.831557,116.328110,1536.815794,...,0.015659,-0.000465,179.605540,179.680329,7.478847e-02,-777.0,0.0,0.000000,0.468110,0.000000


**For each 10-th gps point**

Delta time

In [133]:
%%time

dff['delta_time_10'] = abs(dff['time'] - dff['time'].shift(-10)).apply(lambda x: x.seconds)
dff['lat_10'] = dff['lat'].shift(-10)
dff['lon_10'] = dff['lon'].shift(-10)

Wall time: 2min 12s


Threshold

In [172]:
dff = dff.iloc[:-10]
dff[dff['delta_time_10'] > 66150].shape

(5828, 26)

Distance

In [ ]:
%%time
dff['distance_10'] = dff[['lat','lat_10','lon','lon_10','alt','delta_time_10']].apply(compute_distance_10, axis=1)

Speed

In [ ]:
%%time
dff['speed_10'] = dff[['distance_10','delta_time_10']].apply(compute_speed_10, axis=1)

Acceleration

In [ ]:
dff['speed_10'] = dff['speed'].shift(-10)
dff = dff.iloc[:-1]

In [ ]:
%%time
dff['acceleration_10'] = dff[['speed','speed_10','delta_time_10']].apply(compute_acceleration_10, axis=1)

Jerk

In [ ]:
%%time
dff['jerk_10'] = dff[['acceleration','acceleration_10','delta_time_10']].apply(compute_jerk_10, axis=1)

Bearing

In [ ]:
%%time
dff['bearing_10'] = dff[['lat','lat_10','lon','lon_10']].apply(compute_bearing_10, axis=1)

Bearing rate

In [ ]:
%%time
dff['bearing_rate_10'] = dff[['bearing_10','bearing']].apply(compute_bearing_rate_10, axis=1)

Elevation change

In [ ]:
dff['alt_10'] = dff['alt'].shift(-10)
dff = dff.iloc[:-10]

In [ ]:
%%time
dff['elevation_change_10'] = dff[['alt_10','alt']].apply(compute_elevation_change_10, axis=1)

Elevation speed

In [ ]:
%%time
dff['elevation_speed_10'] = dff[['elevation_change_10','speed_10']].apply(compute_elevation_speed_10, axis=1)

Elevation angle

In [ ]:
%%time
dff['elevation_angle_10'] = dff[['alt_10','distance_10']].apply(compute_elevation_angle_10, axis=1)

Elevation distance

In [ ]:
%%time
dff['elevation_distance_10'] = dff[['elevation_change_10','distance_10']].apply(compute_elevation_distance_10, axis=1)

In [203]:
dff

,time,lat,lon,alt,label,user,delta_time,lat_1,lon_1,distance,...,speed_10,acceleration_10,jerk_10,bearing_10,bearing_rate_10,alt_10,elevation_change_10,elevation_speed_10,elevation_angle_10,elevation_distance_10
1,2008-03-28 14:54:40,39.894178,116.318200,-777.0,6,10,34.0,39.894505,116.321132,253.375124,...,28.591177,0.037348,0.000050,180.584289,0.387508,-777.0,0.0,0.000000,0.110478,0.000000
2,2008-03-28 14:55:14,39.894505,116.321132,-777.0,6,10,59.0,39.894953,116.326452,457.703187,...,29.530746,0.036779,0.000031,180.105377,0.251844,-777.0,0.0,0.000000,0.089597,0.000000
3,2008-03-28 14:56:13,39.894953,116.326452,-777.0,6,10,59.0,39.894600,116.332542,522.318591,...,31.464909,0.038196,-0.000009,179.531741,0.876661,-777.0,0.0,0.000000,0.074493,0.000000
4,2008-03-28 14:57:12,39.894600,116.332542,-777.0,6,10,59.0,39.889622,116.337040,673.422399,...,31.509769,0.033946,-0.000025,178.891087,1.407696,-777.0,0.0,0.000000,0.063319,0.000000
5,2008-03-28 14:58:11,39.889622,116.337040,-777.0,6,10,59.0,39.882090,116.338353,843.803760,...,29.131200,0.025050,0.000016,178.362414,1.724043,-777.0,0.0,0.000000,0.056979,0.000000
6,2008-03-28 14:59:10,39.882090,116.338353,-777.0,6,10,60.0,39.873873,116.338455,912.394041,...,19.515034,0.007278,-0.000020,178.061995,1.944676,-777.0,0.0,0.000000,0.053372,0.000000
7,2008-03-28 15:00:10,39.873873,116.338455,-777.0,6,10,59.0,39.865182,116.338058,965.577796,...,15.571418,-0.001342,-0.000061,178.274790,1.699422,-777.0,0.0,0.000000,0.052878,0.000000
8,2008-03-28 15:01:09,39.865182,116.338058,-777.0,6,10,59.0,39.855403,116.337710,1086.189677,...,11.119672,-0.012315,-0.000096,178.547551,1.430031,-777.0,0.0,0.000000,0.053724,0.000000
9,2008-03-28 15:02:08,39.855403,116.337710,-777.0,6,10,59.0,39.844532,116.334362,1240.567528,...,9.920764,-0.017656,-0.000163,178.870938,0.915318,-777.0,0.0,0.000000,0.054611,0.000000
10,2008-03-28 15:03:07,39.844532,116.334362,-777.0,6,10,59.0,39.831557,116.328110,1536.815794,...,9.384254,-0.026492,-0.000111,179.249652,0.355888,-777.0,0.0,0.000000,0.058012,0.000000


Remove duplicating columns

In [206]:
dff.columns

Index(['time', 'lat', 'lon', 'alt', 'label', 'user', 'delta_time', 'lat_1',
       'lon_1', 'distance', 'speed', 'speed_1', 'acceleration',
       'acceleration_1', 'jerk', 'bearing', 'bearing_1', 'bearing_rate',
       'alt_1', 'elevation_change', 'elevation_speed', 'elevation_angle',
       'elevation_distance', 'delta_time_10', 'lat_10', 'lon_10',
       'distance_10', 'speed_10', 'acceleration_10', 'jerk_10', 'bearing_10',
       'bearing_rate_10', 'alt_10', 'elevation_change_10',
       'elevation_speed_10', 'elevation_angle_10', 'elevation_distance_10'],
      dtype='object')

In [216]:
dff = dff.iloc[:-10]

data = dff.copy()
data.drop(['lat_1','lon_1','speed_1','acceleration_1', 'bearing_1', 'alt_1',\
           'lat_10', 'lon_10', 'alt_10'], axis=1, inplace=True)
data

,time,lat,lon,alt,label,user,delta_time,distance,speed,acceleration,...,distance_10,speed_10,acceleration_10,jerk_10,bearing_10,bearing_rate_10,elevation_change_10,elevation_speed_10,elevation_angle_10,elevation_distance_10
1,2008-03-28 14:54:40,39.894178,116.318200,-777.0,6,10,34.0,253.375124,7.452210,0.008984,...,7004.449301,28.591177,0.037348,0.000050,180.584289,0.387508,0.0,0.000000,0.110478,0.000000
2,2008-03-28 14:55:14,39.894505,116.321132,-777.0,6,10,59.0,457.703187,7.757681,0.018562,...,8648.936946,29.530746,0.036779,0.000031,180.105377,0.251844,0.0,0.000000,0.089597,0.000000
3,2008-03-28 14:56:13,39.894953,116.326452,-777.0,6,10,59.0,522.318591,8.852857,0.043408,...,10411.256219,31.464909,0.038196,-0.000009,179.531741,0.876661,0.0,0.000000,0.074493,0.000000
4,2008-03-28 14:57:12,39.894600,116.332542,-777.0,6,10,59.0,673.422399,11.413939,0.048946,...,12254.781563,31.509769,0.033946,-0.000025,178.891087,1.407696,0.0,0.000000,0.063319,0.000000
5,2008-03-28 14:58:11,39.889622,116.337040,-777.0,6,10,59.0,843.803760,14.301759,0.015336,...,13621.904751,29.131200,0.025050,0.000016,178.362414,1.724043,0.0,0.000000,0.056979,0.000000
6,2008-03-28 14:59:10,39.882090,116.338353,-777.0,6,10,60.0,912.394041,15.206567,0.019319,...,14544.387148,19.515034,0.007278,-0.000020,178.061995,1.944676,0.0,0.000000,0.053372,0.000000
7,2008-03-28 15:00:10,39.873873,116.338455,-777.0,6,10,59.0,965.577796,16.365725,0.034649,...,14680.422099,15.571418,-0.001342,-0.000061,178.274790,1.699422,0.0,0.000000,0.052878,0.000000
8,2008-03-28 15:01:09,39.865182,116.338058,-777.0,6,10,59.0,1086.189677,18.409995,0.044349,...,14448.932045,11.119672,-0.012315,-0.000096,178.547551,1.430031,0.0,0.000000,0.053724,0.000000
9,2008-03-28 15:02:08,39.855403,116.337710,-777.0,6,10,59.0,1240.567528,21.026568,0.085104,...,14213.822264,9.920764,-0.017656,-0.000163,178.870938,0.915318,0.0,0.000000,0.054611,0.000000
10,2008-03-28 15:03:07,39.844532,116.334362,-777.0,6,10,59.0,1536.815794,26.047725,0.043109,...,13378.801666,9.384254,-0.026492,-0.000111,179.249652,0.355888,0.0,0.000000,0.058012,0.000000


**Save to CSV**

In [217]:
%%time
data.to_csv('features.csv', index=False)

Wall time: 15min 41s


**Features examples**

In [223]:
data.iloc[15666]

time                     2008-04-03 18:23:00
lat                                  41.8338
lon                                  85.9166
alt                                     -777
label                                      6
user                                      10
delta_time                                 1
distance                             27.7816
speed                                27.7816
acceleration                      -0.0704794
jerk                                0.275015
bearing                              359.978
bearing_rate                     6.57481e-05
elevation_change                           0
elevation_speed                            0
elevation_angle                      1.53506
elevation_distance                         0
delta_time_10                             10
distance_10                          278.089
speed_10                             28.0895
acceleration_10                    0.0307898
jerk_10                            0.0101269
bearing_10